<a href="https://colab.research.google.com/github/ctello-austral/labo2025v/blob/main/CT_Optimizacion_Bayesiana_para_Arboles_Azarosos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
principio <- Sys.time()
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# Cargar paquetes
library(ParBayesianOptimization)
# cargo las librerias que necesito
require("data.table")
require("rpart")

In [ ]:
setwd("/content/buckets/b1/exp")
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [ ]:
# Cargar datos y preparar entorno
set.seed(310019)
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]
campos_buenos <- setdiff(colnames(dtrain), "clase_ternaria")

# 1. Modificación de la Función Objetivo
# Se añade 'porc_columnas' como argumento.
funcion_objetivo <- function(minsplit, minbucket, maxdepth, feature_fraction) {
  resultados <- c()

  for (i in 1:5) {
    # Usar el argumento 'porc_columnas'
    campos_random <- sample(campos_buenos, size = as.integer(length(campos_buenos) * feature_fraction))
    formula <- as.formula(paste("clase_ternaria ~", paste(campos_random, collapse = " + ")))

    # ... (el resto del código del rpart es el mismo) ...

    modelo <- rpart(
      formula = formula,
      data = dtrain,
      xval = 0,
      control = rpart.control(
        cp = -1,
        minsplit = as.integer(minsplit),
        minbucket = as.integer(minbucket),
        maxdepth = as.integer(maxdepth)
      )
    )

    pred <- predict(modelo, dfuture, type = "prob")[, "BAJA+2"]
    resultados <- c(resultados, sum(pred, na.rm = TRUE))
  }

  return(list(Score = mean(resultados)))
}

# 2. Modificación del Llamado a bayesOpt
# Se añade 'porc_columnas' a los 'bounds'
optimizacion <- bayesOpt(
  FUN = funcion_objetivo,
  bounds = list(
    minsplit = c(500L, 15000L),
    minbucket = c(10L, 5000L),
    maxdepth = c(4L, 30L),
    feature_fraction = c(0.1, 0.9)
  ),
  initPoints = 10,
  iters.n = 20,
  acq = "ucb",
  kappa = 2.576,
  verbose = 1
)

In [ ]:
# Mejores hiperparámetros y métrica
# Mostrar mejores hiperparámetros
print(getBestPars(optimizacion))